In [1]:
!pip install torch
!pip install gensim
!pip install nltk
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install transformers

!pip install keras
!pip install tensorflow
import keras
import tensorflow



import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
# from torch.optim import AdamW
from transformers import BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KDTree
import pandas as pd
import numpy as np
import json
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re




Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


2023-11-24 00:56:10.415192: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 00:56:10.485355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 00:56:11.645033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
df1=pd.read_csv('./df1.csv',nrows=10000)
df1['abstract'] = df1['abstract'].astype(str)
df1.head(10)

In [5]:

# 加载Word2Vec模型
model_path = "./GoogleNews-vectors-negative300.bin"
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)



In [6]:
# 确保这是你的nltk_data目录所在的路径
nltk_data_path = 'autodl-fs/nltk_data2'

# 添加到NLTK的数据路径
nltk.data.path.append(nltk_data_path)

nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to autodl-fs/nltk_data2...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to autodl-fs/nltk_data2...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to autodl-fs/nltk_data2...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:

# 分词和预处理函数
def tokenize_and_process(text):
    words = word_tokenize(text)
    words = [word for word in words if word.isalpha()]
    words = [word.lower() for word in words]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words


# 对DataFrame中的abstract列进行分词和预处理
df1['tokenized_abstract'] = df1['abstract'].apply(tokenize_and_process)

# 函数来将单词转换为Word2Vec向量，如果模型中没有该词，则使用UNK向量
def word_to_vec(word, model):
    return model[word] if word in model.key_to_index else model['UNK']

# 函数来将句子的分词列表转换为Word2Vec向量列表
def tokens_to_vectors(tokens, model):
    return [word_to_vec(token, model) for token in tokens]

# 应用函数将tokenized_abstract列的分词列表转换为Word2Vec向量列表
df1['word_vectors'] = df1['tokenized_abstract'].apply(lambda tokens: tokens_to_vectors(tokens, word2vec_model))


In [8]:


# 找出word_vectors中最长的长度
MAX_SEQUENCE_LENGTH = df1['word_vectors'].apply(len).max()
UNK_VECTOR = word2vec_model['UNK']  # 获取'UNK'的向量表示
# 使用这个长度作为填充或截断的基础
def pad_or_truncate_vectors(word_vectors, max_length, unk_vector):
    """
    如果word_vectors长度小于max_length，则用unk_vector填充；
    如果word_vectors长度大于max_length，则截断。
    """
    # 获取当前word vectors的长度
    sequence_length = len(word_vectors)
    
    # 如果当前长度小于最大长度，进行填充
    if sequence_length < max_length:
        padding = [unk_vector] * (max_length - sequence_length)
        word_vectors.extend(padding)
    # 如果当前长度大于最大长度，进行截断
    elif sequence_length > max_length:
        word_vectors = word_vectors[:max_length]
    
    return word_vectors

# 应用函数pad_or_truncate_vectors到每一行的word_vectors列
df1['padded_word_vectors'] = df1['word_vectors'].apply(
    lambda x: pad_or_truncate_vectors(x, MAX_SEQUENCE_LENGTH, UNK_VECTOR))

# 检查结果
df1['padded_word_vectors'].apply(len)  # 每个向量的长度都应该是MAX_SEQUENCE_LENGTH

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 准备特征：将padded_word_vectors的列表转换为NumPy数组
X = np.array(df1['padded_word_vectors'].tolist())

# 准备标签：获取所有标签列
label_columns = df1.columns.difference(['abstract', 'tokenized_abstract', 'word_vectors', 'padded_word_vectors'])
y = df1[label_columns].values

# 分割数据集为训练集和测试集，这里使用20%的数据作为测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 输出分割后的数据集维度，仅用于确认
print(f'Training set shape: {X_train.shape, y_train.shape}')
print(f'Test set shape: {X_test.shape, y_test.shape}')


Training set shape: ((8000, 217, 300), (8000, 145))
Test set shape: ((2000, 217, 300), (2000, 145))


In [ ]:
from keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam

# 指定学习率
learning_rate = 1e-3

# 创建优化器实例
optimizer = Adam(learning_rate=learning_rate)


def custom_accuracy(y_true, y_pred):
    # 预测值大于0.3的被认为是正类
    threshold = 0.3
    y_pred_thresholded = tf.cast(tf.greater(y_pred, threshold), tf.float32)
    
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred_thresholded), tf.float32))
    return accuracy






def micro_f1(y_true, y_pred):
    # 预测值大于0.3的被认为是正类
    y_pred = K.cast(K.greater(y_pred, 0.3), K.floatx())
    
    # 计算真正例、假正例和假负例
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    
    # 计算精确度和召回率
    precision = K.sum(true_positives) / (K.sum(predicted_positives) + K.epsilon())
    recall = K.sum(true_positives) / (K.sum(possible_positives) + K.epsilon())
    
    # 计算micro-F1分数
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1


# 模型参数
max_sequence_length = 237  # 句子的最大长度
embedding_dim = 300  # 词嵌入的维度
num_labels = 145  # 标签的数量

# 模型输入
input_ = Input(shape=(max_sequence_length, embedding_dim))

# 卷积层：使用多个不同大小的滤波器来提取特征
# 可以通过交叉验证调整filters和kernel_size
conv = Conv1D(filters=128, kernel_size=5, activation='relu')(input_)

# 全局最大池化层：为了捕捉最重要的特征
gmp = GlobalMaxPooling1D()(conv)

# Dropout层：减少过拟合
dropout = Dropout(0.5)(gmp)

# 全连接层：进一步处理特征
# 可以通过交叉验证调整units
dense = Dense(128, activation='relu')(dropout)

# 输出层：使用sigmoid函数，适用于多标签分类
output = Dense(num_labels, activation='sigmoid')(dense)

# 构建模型
model = Model(inputs=input_, outputs=output)

# 编译模型：使用二元交叉熵作为损失函数，适用于多标签分类
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[micro_f1])

# 打印模型结构
print(model.summary())

# 训练模型：使用训练数据和验证数据
# 这里假设X_train和y_train已经准备好，且与模型输入输出匹配
# 可以通过交叉验证调整batch_size和epochs
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.1)



In [ ]:
# 在测试集上评估模型性能
test_loss, test_micro_f1 = model.evaluate(X_test, y_test)

# 打印测试集上的性能
print(f'Test loss: {test_loss}')
print(f'Test loss: {test_micro_f1}')


In [12]:

from sklearn.model_selection import RandomizedSearchCV
import numpy as np

from keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam
# 指定学习率

def micro_f1(y_true, y_pred):
    # 预测值大于0.3的被认为是正类
    y_pred = K.cast(K.greater(y_pred, 0.3), K.floatx())
    
    # 计算真正例、假正例和假负例
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    
    # 计算精确度和召回率
    precision = K.sum(true_positives) / (K.sum(predicted_positives) + K.epsilon())
    recall = K.sum(true_positives) / (K.sum(possible_positives) + K.epsilon())
    
    # 计算micro-F1分数
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1


# 模型参数
max_sequence_length = 217  # 句子的最大长度
embedding_dim = 300  # 词嵌入的维度
num_labels = 145  # 标签的数量


# 构建模型的函数
def build_model(learning_rate=1e-3, filters=128, kernel_size=5, dense_units=128):
    input_ = Input(shape=(max_sequence_length, embedding_dim))
    conv = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(input_)
    gmp = GlobalMaxPooling1D()(conv)
    dropout = Dropout(0.5)(gmp)
    dense = Dense(dense_units, activation='relu')(dropout)
    output = Dense(num_labels, activation='sigmoid')(dense)
    
    model = Model(inputs=input_, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=[micro_f1])
    
    return model



# 定义要搜索的超参数网格
param_grid = {
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'filters': [64, 128, 256],
    'kernel_size': [3, 5, 7],
    'dense_units': [64, 128, 256],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30]
}

# 要搜索的超参数的数量
n_iter =10  # 你可以根据需要进行更多或更少的迭代

best_score = 0
best_params = {}

for i in range(n_iter):
    # 随机选择超参数
    params = {k: np.random.choice(v) for k, v in param_grid.items() if k not in ['batch_size', 'epochs']}

    # 将 kernel_size 参数转换为元组
    if 'kernel_size' in params:
        params['kernel_size'] = (params['kernel_size'],)

    batch_size = np.random.choice(param_grid['batch_size'])
    epochs = np.random.choice(param_grid['epochs'])
    print(f"Testing parameters: {params}, batch_size: {batch_size}, epochs: {epochs}")
    # 构建和编译模型
    model = build_model(**params)
    
    # 训练模型
    history = model.fit(X_train, y_train, 
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=0.1,
                        verbose=0)
    
    # 获取验证集上的micro_f1得分
    val_micro_f1 = np.max(history.history['val_micro_f1'])  # 假设'micro_f1'是编译模型时使用的名称
    
    # 如果找到了更好的得分，更新最佳得分和参数
    if val_micro_f1 > best_score:
        best_score = val_micro_f1
        best_params = params
        best_params['batch_size'] = batch_size
        best_params['epochs'] = epochs
        print(f"New best score: {best_score}")

print(f"Best score: {best_score}")
print(f"Best parameters: {best_params}")


Testing parameters: {'learning_rate': 0.001, 'filters': 128, 'kernel_size': (3,), 'dense_units': 64}, batch_size: 64, epochs: 30
Testing parameters: {'learning_rate': 0.0001, 'filters': 64, 'kernel_size': (5,), 'dense_units': 256}, batch_size: 128, epochs: 10
Testing parameters: {'learning_rate': 0.001, 'filters': 64, 'kernel_size': (3,), 'dense_units': 128}, batch_size: 64, epochs: 20
Testing parameters: {'learning_rate': 0.001, 'filters': 128, 'kernel_size': (7,), 'dense_units': 256}, batch_size: 128, epochs: 20
Testing parameters: {'learning_rate': 0.01, 'filters': 64, 'kernel_size': (3,), 'dense_units': 64}, batch_size: 64, epochs: 20
Best score: 0
Best parameters: {}
